In [1]:
import requests
from requests import Session
import secrets
import json
from pprint import pprint as pp

In [ ]:
import requests
import json
import csv

Settings = json.load(open('settings.json'))


def getAuthToken(user, pw):
  r = requests.post('https://www.echomtg.com/api/user/auth/',
                    data={'email': str(user), 'password': str(pw)})
  responseBody = json.loads(r.text)
  if responseBody['status'] == 'success':
    return responseBody['token']
  else:
    print('Authentication error')
    print(r.text)


def buildCardReferenceList(token):
  cardRequest = requests.get(
      'https://www.echomtg.com/api/stores/card_reference/auth=' + str(token))
  cardResponse = cardRequest.json()
  cardList = cardResponse['cards']
  return cardList


def getNewPrices(token):
  priceRequest = requests.get(
      'https://www.echomtg.com/api/stores/price_reference/auth=' + str(token))
  priceResponse = priceRequest.json()
  priceList = priceResponse['suggested_prices']
  return priceList


def outputCSV(reference, prices):
  with open('priceUpdate.csv', 'w') as csvfile:
    rowwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    rowwriter.writerow(['Product Name', 'Set Name', 'Sell Price'])
    for card, data in prices.items():
      rowwriter.writerow([reference[str(card)]['name'],
                          reference[str(card)]['expansion'], data['price']])


AuthToken = getAuthToken(Settings['username'], Settings['password'])

cardReference = buildCardReferenceList(AuthToken)

priceList = getNewPrices(AuthToken)

outputCSV(cardReference, priceList)

In [21]:
url = 'https://www.echomtg.com/api/inventory/dump/'


# start=0&limit=100&auth=c07936c0e033032d4e220ab74f0b99a1c60061a8
# parameters to give to request
#parameters = {
#  'start':'0',
#  'limit':'100',
#}

# qualifiers

# initialize connection
session = Session()


session.headers
#actually attempting to get the request
try:
  response = session.get(url + "auth=c07936c0e033032d4e220ab74f0b99a1c60061a8") #params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': 'success', 'showDeleted': False, 'showSince': 'none', 'accessDate': '2022-07-12T12:19:47-04:00', 'headerMap': {'i': 'ID (EchoMTG Unique Inventory ID)', 'e': 'EMID  (EchoMTG Item/Card ID)', 't': 'Tradable', 'f': 'Foil', 'p': 'Price', 'a': 'Date Acquired', 'c': 'Created Date', 'u': 'Last Updated Date', 'd': 'Deleted Date'}, 'inventoryData': [{'i': '45109693', 'e': '138501', 't': '0', 'f': '1', 'p': '10.00', 'a': '2022-07-12 13:10:13', 'c': '2022-07-12 13:10:13', 'u': '2022-07-12 13:10:13', 'd': None}, {'i': '45049212', 'e': '138598', 't': '0', 'f': '1', 'p': '45.00', 'a': '2022-07-09 12:54:52', 'c': '2022-07-09 12:54:52', 'u': '2022-07-09 12:54:52', 'd': None}, {'i': '45049210', 'e': '138429', 't': '0', 'f': '1', 'p': '45.00', 'a': '2022-07-09 12:54:34', 'c': '2022-07-09 12:54:34', 'u': '2022-07-09 12:54:34', 'd': None}, {'i': '45049209', 'e': '138429', 't': '0', 'f': '0', 'p': '45.00', 'a': '2022-07-09 12:54:33', 'c': '2022-07-09 12:54:33', 'u': '2022-07-09 12:54:33', 'd': No

In [19]:
respon = session.get(url)

In [20]:
respon.text

'{"message":"Not authorized to make this change. Please authorize see the API documentation to learn how to authorize.","status":"error"}'

In [ ]:
class echoMTGAPI:
    
    def __init__(self, token):
        self.api_url = 'https://www.echomtg.com/'
        self.headers = {'Accepts': 'application/json', 'X-CMC_PRO_API_KEY': token,}
        self.session = Session()
        # cannot use the headers = self.headers here, since there is a secret splat
        self.session.headers.update(self.headers)

        
        
# need to get inventory!

# that's about it


##
# https://www.echomtg.com/api/inventory/view/start=0&limit=100&sort=price&order=desc&auth=c07936c0e033032d4e220ab74f0b99a1c60061a8
##

    def getAllCoins(self):
        url = self.api_url + "v1/cryptocurrency/map"
        r = self.session.get(url)
        data = r.json()['data']
        return data
    
    def latestPrice(self, coinCodes = "BTC"):
        url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
        parameters = {
            'symbol': coinCodes
        }
        r = self.session.get(url, params = parameters)
        data = r.json()['data']
        return data